In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)
mnist

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f5565817198>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f55b1570518>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f55657b5f60>)

In [3]:
sess = tf.InteractiveSession()

In [4]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [5]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y),
                                              reduction_indices=[1]))

In [6]:
train_step = tf.train.GradientDescentOptimizer(0.5) \
    .minimize(cross_entropy)
    
def do_train_steps(train_step = train_step, 
                   mnist=mnist, 
                   n=1001,
                   batch_size=50):
    
    for i in range(n):
        
        batch = mnist.train.next_batch(batch_size)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})
        
        if i % 200 == 0:
            correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            print("Step [%d], acc = %.1f%%" % (
                    i,
                    100. * accuracy.eval(feed_dict = {x: mnist.test.images,
                                                      y_: mnist.test.labels})))

In [7]:
do_train_steps(n=1001)

Step [0], acc = 28.5%
Step [200], acc = 89.5%
Step [400], acc = 90.3%
Step [600], acc = 91.1%
Step [800], acc = 89.0%
Step [1000], acc = 90.9%


In [8]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

n_hidden = 200

W1 = tf.Variable(tf.zeros([784, n_hidden]))
b1 = tf.Variable(tf.zeros([n_hidden]))

W2 = tf.Variable(tf.zeros([n_hidden, n_hidden]))
b2 = tf.Variable(tf.zeros([n_hidden]))

W3 = tf.Variable(tf.zeros([n_hidden, 10]))
b3 = tf.Variable(tf.zeros([10]))

sess.run(tf.initialize_all_variables())

y1 = tf.nn.dropout(tf.nn.sigmoid(tf.matmul(x, W1) + b1),  keep_prob=0.5)
y2 = tf.nn.dropout(tf.nn.sigmoid(tf.matmul(y1, W2) + b2),  keep_prob=0.5)
y3 = tf.nn.softmax(tf.matmul(y2, W3) + b3)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y3),
                                              reduction_indices=[1]))

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [9]:
train_step = tf.train.GradientDescentOptimizer(0.5) \
    .minimize(cross_entropy)
    
def do_train_steps(train_step = train_step, 
                   mnist=mnist, 
                   n=1001,
                   batch_size=50):
    
    c = 32
    
    for i in range(n):
        
        batch = mnist.train.next_batch(batch_size)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})
        
        if i >= c:
            c *= 2
            correct_prediction = tf.equal(tf.argmax(y3, 1), tf.argmax(y_, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            print("Step [%d], acc = %.1f%%" % (
                    i,
                    100. * accuracy.eval(feed_dict = {x: mnist.test.images,
                                                      y_: mnist.test.labels})))

In [10]:
%time do_train_steps(n=2**15+1)

Step [32], acc = 11.0%
Step [64], acc = 11.2%
Step [128], acc = 12.5%
Step [256], acc = 20.4%
Step [512], acc = 31.4%
Step [1024], acc = 49.9%
Step [2048], acc = 83.2%
Step [4096], acc = 89.1%
Step [8192], acc = 92.8%
Step [16384], acc = 94.6%
Step [32768], acc = 95.4%
CPU times: user 1min 41s, sys: 7.4 s, total: 1min 48s
Wall time: 1min 12s
